In [1]:
import re
import dill
from requests_futures.sessions import FuturesSession
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

# Scrape race pages

In [17]:
race_ids = list(range(1, 13000))
session = FuturesSession(max_workers=8)
futures = [session.get(f'https://results.bikereg.com/race/{i}') for i in race_ids]

In [18]:
for i, future in enumerate(futures):
    dill.dump(future.result().text, open(f'C:/data/results/{i}.pkd', 'wb'))

## Extract race metadata

In [20]:
def get_data(text):
    name = ''
    date = ''
    loc = ''
    json_url = ''
    weather = ''
    strava_url = ''
    coord = None
    
    if 'No data' not in text:
        soup = BeautifulSoup(text)
        resultstitle = soup.select('div#resultstitle')[0].text
        split = resultstitle.split('•')

        regex = re.compile(r'([A-Za-z]{3})\s+(\d{1,2})\s+(\d{4})')
        name = split[0].strip()
        month, day, year = regex.findall(split[1].strip())[0]
        date = datetime.strptime(' '.join([month, day, year]), '%b %d %Y')
        loc = ''
        if len(split) > 2:
            loc = re.split('[\t\n\r]', split[2])[0].strip()
            
        json_url = soup.select('span.downloadoptions')[0].find_all('a')[1]['href']
        weather = soup.select('div#conditionssummary')[0].text.strip()
        strava_url = soup.select('div#stravasegment')[0].select('a')[1]['href']
        
        try:
            coord = re.findall(r'GetMap\("([-\d\.]+):([-\d\.]+)"', text)[0]
        except IndexError:
            pass
            
    return name, date, loc, json_url, weather, strava_url, coord

In [21]:
columns=['name', 'date', 'loc', 'json_url', 'weather', 'strava_url', 'coord']
df = pd.DataFrame(columns=columns)
for i in race_ids:  # loop takes ~2 hour or so
    if i % 100 == 0:
        print(i)
    if i in [12533, 12534]:  # these broke for some reason >:|
        continue
    text = dill.load(open(f'C:/data/results/{i}.pkd', 'rb'))
    data = get_data(text)
    df.loc[i] = {c: d for c, d in zip(columns, data)}

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900


FileNotFoundError: [Errno 2] No such file or directory: 'C:/data/results/12999.pkd'

In [22]:
df.to_pickle('C:/data/results/df.pkl')

## reload

In [2]:
df = pd.read_pickle('C:/data/results/df.pkl')
df = df[df['json_url'] != '']
df = df.shift(1).drop(index=1)  # index was shifted for some reason
df['weather'] = df['weather'].str.split('\n').str.get(-1)  # sometimes the weather column also got "Series: " information

In [3]:
df.sample(5)

,name,date,loc,json_url,weather,strava_url,coord
894,Highland Rim - Warren County Commission Time T...,2011-05-14 00:00:00,,downloadrace.php?raceID=894&json=1,,http://app.strava.com/segments/,None
3014,Joe Martin Stage Race - Crit,2013-04-28 00:00:00,"Fayetteville, AR",downloadrace.php?raceID=3014&json=1,"Clear, 62 degrees, wind 5 mph",http://app.strava.com/segments/,None
10059,Grafton Notch Challenge,2018-04-28 00:00:00,"Bethel, ME",downloadrace.php?raceID=10059&json=1,"Clear, 57 degrees, wind 5 mph",http://app.strava.com/segments/9448198,None
9601,Green Mountain Stage Race Day 4 Crit,2017-09-04 00:00:00,"Burlington, VT",downloadrace.php?raceID=9601&json=1,"Clear, 69 degrees, wind 6 mph",http://app.strava.com/segments/,None
12007,Citrus Twilights,2019-07-19 00:00:00,"Redlands, CA",downloadrace.php?raceID=12007&json=1,"Partly Cloudy, 77 degrees, wind 5 mph",http://app.strava.com/segments/,"(34.0553817749023, -117.182586669922)"


In [4]:
df.loc[3635]

name          Greenbelt Park Training Race Series
date                          2013-08-21 00:00:00
loc                                 Greenbelt, MD
json_url      downloadrace.php?raceID=3635&json=1
weather            Clear, 84  degrees, wind 7 mph
strava_url        http://app.strava.com/segments/
coord                                        None
Name: 3635, dtype: object

## Download JSON results

In [6]:
urls = df['json_url']
urls.head()

2    downloadrace.php?raceID=2&json=1
3    downloadrace.php?raceID=3&json=1
4    downloadrace.php?raceID=4&json=1
5    downloadrace.php?raceID=5&json=1
6    downloadrace.php?raceID=6&json=1
Name: json_url, dtype: object

In [104]:
for i, url in urls[:10].iteritems():
    print(i, url)
    full_url = f'https://results.bikereg.com/{url}'
    df_race = pd.read_json(full_url)
    columns = [str(i) for i in range(28)]  # Each column repeated as an "index" for some reason
    df_race = df_race.drop(columns=columns)

2 downloadrace.php?raceID=2&json=1
3 downloadrace.php?raceID=3&json=1
4 downloadrace.php?raceID=4&json=1
5 downloadrace.php?raceID=5&json=1
6 downloadrace.php?raceID=6&json=1
7 downloadrace.php?raceID=7&json=1
8 downloadrace.php?raceID=8&json=1
9 downloadrace.php?raceID=9&json=1
10 downloadrace.php?raceID=10&json=1
11 downloadrace.php?raceID=11&json=1


In [106]:
df_race.head()

,RaceName,RaceCategoryName,Place,RacerID,FirstName,LastName,TeamName,RaceTime,Points,PriorPoints,...,TeamID,Category,Starters,ReportedAge,CalculatedAge,RacerCount,FinishPhotoUrl,License,MetaDataUrl,ResultID
0,Blount Seafoods Fall River Criterium,CAT 3 MEN,1,1693,Bill,Yabroudy,NBX/Narragansett Beer,0:51:22.43,242.776661,268.041288,...,1003,3.0,NaN,NaN,41.0,34,NaN,551281,NaN,1825
1,Blount Seafoods Fall River Criterium,CAT 3 MEN,2,337,David,Fierro,Exodus Road Racing/Pools etc/Br,0:51:24.73,246.142740,276.955380,...,1611,2.0,NaN,NaN,28.0,34,NaN,242328,NaN,1826
2,Blount Seafoods Fall River Criterium,CAT 3 MEN,3,2174,Nick,Pignatello iii,Exodus Road Racing,0:51:30.17,249.508819,287.231062,...,1099,3.0,NaN,NaN,29.0,34,NaN,240810,NaN,1827
3,Blount Seafoods Fall River Criterium,CAT 3 MEN,4,653,Ernest,Tautkus,Exodus Road Racing/Pools etc/Br,0:51:33.50,252.874897,292.672362,...,1611,2.0,NaN,NaN,36.0,34,NaN,107000,NaN,1828
4,Blount Seafoods Fall River Criterium,CAT 3 MEN,5,678,Morgan,Hiller,CLNoonan/KAM/Coast to Coast,,256.240976,321.448077,...,1174,3.0,NaN,NaN,20.0,34,NaN,216869,NaN,1829


## Get all races

In [16]:
session = FuturesSession(max_workers=8)
futures = [session.get(f'https://results.bikereg.com/{url}') for url in urls]

In [17]:
for i, future in enumerate(futures):
    dill.dump(future.result().text, open(f'C:/data/results/races/{i}.pkd', 'wb'))

## Load all races into individual dataframes and merge with original df

In [1]:
import glob
import re
import dill
import pandas as pd

In [2]:
df = pd.read_pickle('C:/data/results/df.pkl')
df = df[df['json_url'] != '']
df = df.shift(1).drop(index=1)  # index was shifted for some reason
df['weather'] = df['weather'].str.split('\n').str.get(-1)  # sometimes the weather column also got "Series: " information
df.sample(5)

,name,date,loc,json_url,weather,strava_url,coord
1647,Collegiate Road Nationals - TTT,2012-05-04 00:00:00,,downloadrace.php?raceID=1647&json=1,"Partly Cloudy, 57 degrees, wind 7 mph",http://app.strava.com/segments/,None
6914,Cal Aggie Criterium,2016-01-30 00:00:00,"Sacramento, CA",downloadrace.php?raceID=6914&json=1,"Partly Cloudy, 56 degrees, wind 2 mph",http://app.strava.com/segments/585888,None
4803,Intelligentsia Cup Prairie State Cycling Serie...,2014-07-27 00:00:00,"Downers Grove, IL",downloadrace.php?raceID=4803&json=1,"Clear, 84 degrees, wind 17 mph",http://app.strava.com/segments/,None
1380,Carondelicious Criterium,2012-03-10 00:00:00,,downloadrace.php?raceID=1380&json=1,"Clear, 56 degrees, wind 9 mph",http://app.strava.com/segments/,None
2819,Long Beach Island TT,2013-04-20 00:00:00,"Long Beach Township, NJ",downloadrace.php?raceID=2819&json=1,"Partly Cloudy, 53 degrees, wind 14 mph",http://app.strava.com/segments/,None


In [3]:
columns = [str(i) for i in range(28)]  # Each column repeated for some reason
for file in glob.iglob(r'C:\data\results\races\*.pkd'):    
    json = dill.load(open(file, 'rb'))

    df_race = pd.read_json(json)
    if df_race.empty:
        continue
    
    # drop duplicate columns
    df_race = df_race.drop(columns=columns)

    race_id = re.findall('(\d+)', file)[0]
    df_race['race_id'] = int(race_id) + 2 # somehow offset by 2
    dfasdfas

NameError: name 'dfasdfas' is not defined

In [5]:
df_race.sample(5)

,RaceName,RaceCategoryName,Place,RacerID,FirstName,LastName,TeamName,RaceTime,Points,PriorPoints,...,Category,Starters,ReportedAge,CalculatedAge,RacerCount,FinishPhotoUrl,License,MetaDataUrl,ResultID,race_id
42,NCC 10th Tour of the Hilltowns,Cat 4,43,734,Mark,Rancourt,noreast cycling,st,389.787234,394.545455,...,4.0,NaN,NaN,35.0,94,NaN,191314,NaN,1306,2
284,NCC 10th Tour of the Hilltowns,Men Cat 3,16,625,Daniel,Watts,CAMBRIDGE BICYCLES,st,280.088759,300.000000,...,3.0,NaN,NaN,28.0,75,NaN,156355,NaN,1181,2
123,NCC 10th Tour of the Hilltowns,cat 5,30,473,Cal,Lankton,,0:22:25,532.040816,500.000000,...,NaN,NaN,NaN,NaN,43,NaN,None,NaN,959,2
161,NCC 10th Tour of the Hilltowns,Men 40+ (Cat 1-,25,578,David,Foley,BOB-Stonyfield Farm-Goodale's Bike Shop,st,313.787717,359.976184,...,3.0,NaN,NaN,47.0,63,NaN,11937,NaN,1266,2
382,NCC 10th Tour of the Hilltowns,Pro 1/2/3,39,522,Duncan,Mcgovern,NorEast Cycling,0:40:58,221.310269,199.158939,...,2.0,NaN,NaN,20.0,48,NaN,203374,NaN,1207,2
